In [1]:
# !pip install mecab-python3

# !pip install unidic-lite
# !pip install japanize-matplotlib

In [1]:
import MeCab
import unidic
import sys
import re
import unicodedata
import matplotlib.pyplot as plt
import tqdm
import japanize_matplotlib
import os
import collections
import glob
import pandas as pd

In [2]:
# Check if MeCab works
# t = MeCab.Tagger('-r/nul -d"C:/Program Files (x86)/MeCab/dic/ipadic"')

t = MeCab.Tagger('-r/nul -d"C:/Program Files (x86)/MeCab/dic/ipadic"')

sentence = 'すもももももももものうち'
print(t.parse(sentence))

すもも	名詞,一般,*,*,*,*,すもも,スモモ,スモモ
も	助詞,係助詞,*,*,*,*,も,モ,モ
もも	名詞,一般,*,*,*,*,もも,モモ,モモ
も	助詞,係助詞,*,*,*,*,も,モ,モ
もも	名詞,一般,*,*,*,*,もも,モモ,モモ
の	助詞,連体化,*,*,*,*,の,ノ,ノ
うち	名詞,非自立,副詞可能,*,*,*,うち,ウチ,ウチ
EOS



### Here is a translation 
- すもも	名詞 (= noun),一般 (=general),*,*,*,*,すもも,スモモ,スモモ
- も	助詞 (=particle) ,係助詞 (=binding particle),*,*,*,*,も,モ,モ
- もも	名詞 (= noun),一般 (=general),*,*,*,*,もも,モモ,モモ
- も	助詞 (=particle) ,係助詞 (=binding particle),*,*,*,*,も,モ,モ
- もも	名詞 (= noun),一般 (=general),*,*,*,*,もも,モモ,モモ
- の	助詞 (=particle) ,連体化(=attributive),*,*,*,*,の,ノ,ノ
- うち	名詞(=noun),非自立(=non-indipendent),副詞可能(adverb)=,*,*,*,うち,ウチ,ウチ

In [3]:
#create the list of meaningless words such as "that" "year" "what" etc... 
with open("stopwords.txt", encoding='UTF-8') as f:
    stopwords = f.read().split("\n")
print(stopwords)

['あそこ', 'あたり', 'あちら', 'あっち', 'あと', 'あな', 'あなた', 'あれ', 'いくつ', 'いつ', 'いま', 'いや', 'いろいろ', 'うち', 'おおまか', 'おまえ', 'おれ', 'がい', 'かく', 'かたち', 'かやの', 'から', 'がら', 'きた', 'くせ', 'ここ', 'こっち', 'こと', 'ごと', 'こちら', 'ごっちゃ', 'これ', 'これら', 'ごろ', 'さまざま', 'さらい', 'さん', 'しかた', 'しよう', 'すか', 'ずつ', 'すね', 'すべて', 'ぜんぶ', 'そう', 'そこ', 'そちら', 'そっち', 'そで', 'それ', 'それぞれ', 'それなり', 'たくさん', 'たち', 'たび', 'ため', 'だめ', 'ちゃ', 'てん', 'とおり', 'とき', 'どこ', 'どこか', 'ところ', 'どちら', 'どっか', 'どっち', 'どれ', 'なか', 'なかば', 'なに', 'など', 'なん', 'はじめ', 'はず', 'はるか', 'ひと', 'ひとつ', 'ふく', 'ぶり', 'べつ', 'へん', 'ぺん', 'ほう', 'ほか', 'まさ', 'まし', 'まとも', 'まま', 'みたい', 'みつ', 'みなさん', 'みんな', 'もと', 'もの', 'もん', 'やつ', 'よう', 'よそ', 'わけ', 'わたし', '', 'ハイ', '', '', '上', '中', '下', '字', '', '', '年', '月', '日', '時', '分', '秒', '週', '火', '水', '木', '金', '土', '国', '都', '道', '府', '県', '市', '区', '町', '村', '', '', '各', '第', '方', '何', '的', '度', '文', '者', '体', '人', '他', '今', '部', '課', '係', '外', '類', '達', '気', '室', '口', '誰', '用', '界', '会', '首', '別', '話', '私', '屋', '店', '家', '場', '等', '見', '際', '観', 

In [4]:
def count_word(text, pos_tags, stopwords):
    
    words_list = []
    tagger = MeCab.Tagger('-r/nul -d"C:/Program Files (x86)/MeCab/dic/ipadic"')
    
    node = tagger.parseToNode(text)

    while node:
        features = node.feature.split(",")
        base = features[6]
        pos_tag = features[0]
        
        # remove "stopwords"
        if pos_tag in pos_tags and base not in stopwords:
            words_list.append((base, pos_tag))
            

        node = node.next
    count = collections.Counter(words_list)

    return count


In [5]:
file_list = glob.glob("F:/Wikipedia/wikiextractor-master/wikiextractor/prince/*") 

In [6]:
file_list[:5]

['F:/Wikipedia/wikiextractor-master/wikiextractor/prince\\リチャード2世 (イングランド王)',
 'F:/Wikipedia/wikiextractor-master/wikiextractor/prince\\エドワード5世 (イングランド王)',
 'F:/Wikipedia/wikiextractor-master/wikiextractor/prince\\ヘンリー5世 (イングランド王)',
 'F:/Wikipedia/wikiextractor-master/wikiextractor/prince\\エドワード黒太子',
 'F:/Wikipedia/wikiextractor-master/wikiextractor/prince\\ヘンリー8世 (イングランド王)']

In [7]:
# read stopwords
with open("stopwords.txt",encoding='UTF-8') as f:
    stopwords = f.read().split("\n")

pos_tags = ["名詞", "動詞", "形容詞"] 
#  ["名詞", "動詞", "形容詞"]=  ["noun", "verb", "adj"]

In [8]:
dfs = []

In [9]:
for f in file_list:
    with open(f, 'r', encoding='UTF-8') as file:
        text = file.read()
    
    f_name = f.split("\\")[-1]
    
    count = count_word(text, pos_tags, stopwords)
    
    #top100_words = count.most_common(100)
    all_words = count.items()
    
    all_words_df = (
        pd
        .DataFrame(all_words, columns = ['word_tuple', "count"])
        .assign(
            word = lambda x: x["word_tuple"].apply(lambda wt: wt[0]),
            pos = lambda x: x["word_tuple"].apply(lambda wt: wt[1]),
            file_name = f_name
        )
        .sort_values("count", ascending=False)
    )
    #CHANGE PATH!!!!!!!
    full_csv_file = "F:/Wikipedia/wikiextractor-master/wikiextractor/prince/new_csvs/" + f_name + "_words.csv"
    #print(f_name)
    all_words_df.to_csv(full_csv_file, index=None)
    dfs.append(all_words_df)


PermissionError: [Errno 13] Permission denied: 'F:/Wikipedia/wikiextractor-master/wikiextractor/prince\\csv'

In [11]:
overall = (
    pd
    .concat(dfs)
    .groupby(["word", "pos"])
    .pipe(
        lambda grp: pd.DataFrame({
            "total": grp["count"].sum(),
            "articles": grp["file_name"].nunique()
        }))
    .reset_index()
    .sort_values("total", ascending=False)
)

In [12]:
overall[overall.pos == "名詞"].head(60)

,word,pos,total,articles
1749,世,名詞,2470,305
2385,公,名詞,1127,247
6105,王,名詞,864,229
1162,フランス,名詞,793,147
7782,軍,名詞,762,127
6791,結婚,名詞,568,257
1493,ルイ,名詞,393,85
387,イギリス,名詞,392,99
6017,父,名詞,389,182
6828,継承,名詞,385,148


In [13]:
overall[overall.pos == "動詞"].head(60)

,word,pos,total,articles
6161,生まれる,動詞,350,244
2990,受ける,動詞,211,97
7228,行う,動詞,176,82
2747,務める,動詞,150,95
272,もうける,動詞,141,120
6101,率いる,動詞,140,45
1746,与える,動詞,134,79
7513,認める,動詞,102,59
22,いう,動詞,99,53
4365,得る,動詞,95,56


In [14]:
overall[overall.pos == "形容詞"].head(60)

,word,pos,total,articles
210,ない,形容詞,183,92
3479,多い,形容詞,38,22
4259,強い,形容詞,35,22
5957,無い,形容詞,27,21
3496,大きい,形容詞,27,16
8598,高い,形容詞,25,20
8236,長い,形容詞,24,21
7124,若い,形容詞,23,22
5125,新しい,形容詞,23,12
4145,幼い,形容詞,22,20


In [15]:
overall.to_csv("prince_text_overall.csv", index= False)

In [16]:
df= pd.read_csv("prince_text_overall.csv")
df

,word,pos,total,articles
0,世,名詞,2470,305
1,公,名詞,1127,247
2,王,名詞,864,229
3,フランス,名詞,793,147
4,軍,名詞,762,127
...,...,...,...,...
8660,歓楽,名詞,1,1
8661,歓楽街,名詞,1,1
8662,歓談,名詞,1,1
8663,止む,動詞,1,1
